<a href="https://colab.research.google.com/github/timob0/rag/blob/main/RAGFP_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load required libraries

In [101]:
%%capture
!pip install chromadb tqdm fireworks-ai python-dotenv pandas
!pip install sentence-transformers

import fireworks.client
import os
import dotenv
import chromadb
import json
from tqdm.auto import tqdm
import pandas as pd
import random
from google.colab import userdata
from google.colab import drive
import textwrap

In [2]:
# Retrieve Fireworks AI API key
dotenv.load_dotenv()
fireworks.client.api_key = userdata.get('FW_API_KEY')

# Mount Google Drive with the RFP database
drive.mount('/content/drive')

# !ls "/content/drive/My Drive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 archiv		        RAG_demo.ipynb	 rfp_data.xlsx	       RFP_demo.ipynb
 ml-potw-10232023.csv   rfp_data.csv	'RFP_demo (1).ipynb'


A function to get completions from the fireworks LLM

In [27]:
def get_completion(prompt, model=None, max_tokens=50, p_temperature=0):

    fw_model_dir = "accounts/fireworks/models/"

    if model is None:
        model = fw_model_dir + "llama-v3p1-8b-instruct"
    else:
        model = fw_model_dir + model

    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=p_temperature
    )

    return completion.choices[0].text

Test connectivity to the mistral LLM

In [4]:
mistral_llm = "mixtral-8x7b-instruct"

get_completion("[INST]Tell me 2 jokes[/INST]", model=mistral_llm)

" Sure, here are two jokes for you:\n\n1. Why don't scientists trust atoms?\n\nBecause they make up everything!\n\n1. Why did the scarecrow win an award?\n\nBecause he was outstanding"

Read a database with RFP Questions and Answers to augment the text generation using the LLM.


In [131]:
# load dataset from google drive and convert to dataframe
# dataset contains column names
rfp_file = pd.read_csv('/content/drive/My Drive/Colab Notebooks/rfp_data.csv', header=0, delimiter=";")

# remove rows with empty titles or descriptions
rfp_qa = rfp_file.dropna(subset=["Question", "Answer"])
rfp_qa.tail()
# convert dataframe to list of dicts with Question and Answer columns only
rfp_qa_dict = rfp_qa.to_dict(orient="records")

Install Chroma DB and store word embeddings.

In [126]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        batch_embeddings = embedding_model.encode(input)
        return batch_embeddings.tolist()

embed_fn = MyEmbeddingFunction()

# Initialize the chromadb directory, and client.
client = chromadb.PersistentClient(path="./chromadb")

# create collection
client.delete_collection(name=f"rfp-materials-2024")
collection = client.create_collection(name=f"rfp-materials-2024")
# Generate embeddings, and index titles in batches
batch_size = 50

# loop through batches and generated + store embeddings
for i in tqdm(range(0, len(rfp_qa_dict), batch_size)):

    i_end = min(i + batch_size, len(rfp_qa_dict))
    batch = rfp_qa_dict[i : i + batch_size]

    # Replace empty strings (shouldn't happen)
    batch_questions = [str(paper["Question"]) if str(paper["Question"]) != "" else "No Question" for paper in batch]
    batch_answers = [str(paper["Answer"]) if str(paper["Answer"]) != "" else "No Answer" for paper in batch]
    batch_ids = [str(sum(ord(c) + random.randint(1, 10000) for c in paper["Question"])) for paper in batch]
    batch_metadata = [dict(source=paper["Source"],
                           theme=paper['Theme'].strip(),
                           rfp=paper['Original_RFP']
                           )
                           for paper in batch]

    # generate embeddings
    batch_embeddings = embedding_model.encode(batch_questions)

    # upsert to chromadb
    collection.upsert(
        ids=batch_ids,
        metadatas=batch_metadata,
        documents=batch_answers,
        embeddings=batch_embeddings.tolist(),
    )

  0%|          | 0/6 [00:00<?, ?it/s]

Testing the text retriever, this will retrieve output based on the data found in the vector database.

In [ ]:
collection = client.get_or_create_collection(
    name=f"rfp-materials-2024",
    embedding_function=embed_fn
)

retriever_results = collection.query(
    query_texts=["What are ESG factors?"],
    n_results=2,
)

print(retriever_results["documents"])
print(retriever_results["metadatas"])

In [134]:
rfp_qa['Theme'].unique()

array(['General', 'UK Enhanced ', 'Global Enhanced '], dtype=object)

In [ ]:
# @title Enter your RFP Query and hit return. { run: "auto" }
user_query = "Please explain your risk management approach"  # @param {type:"string"}
user_theme = 'General' # @param ['General', 'UK Enhanced ', 'Global Enhanced ']
user_response_count = 5 # @param {type:"slider", min:1, max:5, step:1}
user_response_temp = 0 # @param {type:"slider", min:0, max:1, step:0.1}
user_response_toks = 250 # @param {type:"slider", min:50, max:5000, step:50}

# query for user query
results = collection.query(
    query_texts=[user_query],
    # where={"$or": [{"theme": user_theme.strip()}, {"theme": "General"}]},
    where={"theme": user_theme.strip()},
    n_results=user_response_count,
)

# concatenate titles into a single string
answers = '\n'.join(results['documents'][0])

prompt_template = f'''[INST]

Your main task is to generate RFP_TEXT by summarizing the RFP_ANSWERS.

You should mimic a style similar to the one in the QUESTION.

QUESTION: {user_query}

RFP_ANSWERS: {answers}

RFP_TEXT:

[/INST]
'''

model_response = get_completion(prompt_template, model=mistral_llm, max_tokens=user_response_toks, p_temperature=user_response_temp)

# Print the suggestions.
print("========================================================================")
print("Your query:")
print()
print(user_query)
print("========================================================================")
print()

print("========================================================================")
print("Model generated answer:")
print()
print(textwrap.fill(model_response,120))
print("========================================================================")
print()

print("========================================================================")
print("Reference answers from RFP database:")
for i in range(len(results['ids'][0])):
  mdo = results['metadatas'][0][i]
  print("=============================================================================")
  print("Result  :",i+1)
  print("Theme   :", mdo['theme'])
  print("Document:", mdo['source'])
  print("Used RFP:", mdo['rfp'])

  print()
  print(textwrap.fill(results['documents'][0][i], 120))
print("========================================================================")